# LR

In [71]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel

from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import lit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorSlicer
from pyspark.sql.functions import regexp_replace

import numpy as np
import pandas as pd


In [2]:
train_data = spark.read.text('C:\spark\data\my_data\iris_data.txt')
train_data.head(5)
# type(train_data)


[Row(value='5.1,3.5,1.4,0.2,Iris-setosa'),
 Row(value='4.9,3.0,1.4,0.2,Iris-setosa'),
 Row(value='4.7,3.2,1.3,0.2,Iris-setosa'),
 Row(value='4.6,3.1,1.5,0.2,Iris-setosa'),
 Row(value='5.0,3.6,1.4,0.2,Iris-setosa')]

In [3]:
train_data

DataFrame[value: string]

In [4]:
train_data.show(100)  # 自动编码？？66

+--------------------+
|               value|
+--------------------+
|5.1,3.5,1.4,0.2,I...|
|4.9,3.0,1.4,0.2,I...|
|4.7,3.2,1.3,0.2,I...|
|4.6,3.1,1.5,0.2,I...|
|5.0,3.6,1.4,0.2,I...|
|5.4,3.9,1.7,0.4,I...|
|4.6,3.4,1.4,0.3,I...|
|5.0,3.4,1.5,0.2,I...|
|4.4,2.9,1.4,0.2,I...|
|4.9,3.1,1.5,0.1,I...|
|5.4,3.7,1.5,0.2,I...|
|4.8,3.4,1.6,0.2,I...|
|4.8,3.0,1.4,0.1,I...|
|4.3,3.0,1.1,0.1,I...|
|5.8,4.0,1.2,0.2,I...|
|5.7,4.4,1.5,0.4,I...|
|5.4,3.9,1.3,0.4,I...|
|5.1,3.5,1.4,0.3,I...|
|5.7,3.8,1.7,0.3,I...|
|5.1,3.8,1.5,0.3,I...|
|5.4,3.4,1.7,0.2,I...|
|5.1,3.7,1.5,0.4,I...|
|4.6,3.6,1.0,0.2,I...|
|5.1,3.3,1.7,0.5,I...|
|4.8,3.4,1.9,0.2,I...|
|5.0,3.0,1.6,0.2,I...|
|5.0,3.4,1.6,0.4,I...|
|5.2,3.5,1.5,0.2,I...|
|5.2,3.4,1.4,0.2,I...|
|4.7,3.2,1.6,0.2,I...|
|4.8,3.1,1.6,0.2,I...|
|5.4,3.4,1.5,0.4,I...|
|5.2,4.1,1.5,0.1,I...|
|5.5,4.2,1.4,0.2,I...|
|4.9,3.1,1.5,0.1,I...|
|5.0,3.2,1.2,0.2,I...|
|5.5,3.5,1.3,0.2,I...|
|4.9,3.1,1.5,0.1,I...|
|4.4,3.0,1.3,0.2,I...|
|5.1,3.4,1.5,0.2,I...|
|5.0,3.5,1.

In [5]:
train_data.groupby('_c4').avg().collect()

AnalysisException: "cannot resolve '`_c4`' given input columns: [value];;\n'Aggregate ['_c4], ['_c4]\n+- AnalysisBarrier\n      +- Relation[value#0] text\n"

In [6]:
train_data.collect()

[Row(value='5.1,3.5,1.4,0.2,Iris-setosa'),
 Row(value='4.9,3.0,1.4,0.2,Iris-setosa'),
 Row(value='4.7,3.2,1.3,0.2,Iris-setosa'),
 Row(value='4.6,3.1,1.5,0.2,Iris-setosa'),
 Row(value='5.0,3.6,1.4,0.2,Iris-setosa'),
 Row(value='5.4,3.9,1.7,0.4,Iris-setosa'),
 Row(value='4.6,3.4,1.4,0.3,Iris-setosa'),
 Row(value='5.0,3.4,1.5,0.2,Iris-setosa'),
 Row(value='4.4,2.9,1.4,0.2,Iris-setosa'),
 Row(value='4.9,3.1,1.5,0.1,Iris-setosa'),
 Row(value='5.4,3.7,1.5,0.2,Iris-setosa'),
 Row(value='4.8,3.4,1.6,0.2,Iris-setosa'),
 Row(value='4.8,3.0,1.4,0.1,Iris-setosa'),
 Row(value='4.3,3.0,1.1,0.1,Iris-setosa'),
 Row(value='5.8,4.0,1.2,0.2,Iris-setosa'),
 Row(value='5.7,4.4,1.5,0.4,Iris-setosa'),
 Row(value='5.4,3.9,1.3,0.4,Iris-setosa'),
 Row(value='5.1,3.5,1.4,0.3,Iris-setosa'),
 Row(value='5.7,3.8,1.7,0.3,Iris-setosa'),
 Row(value='5.1,3.8,1.5,0.3,Iris-setosa'),
 Row(value='5.4,3.4,1.7,0.2,Iris-setosa'),
 Row(value='5.1,3.7,1.5,0.4,Iris-setosa'),
 Row(value='4.6,3.6,1.0,0.2,Iris-setosa'),
 Row(value=

In [7]:
train_data = spark.read.csv('C:\spark\data\my_data\iris_data.csv', header=False)
train_data.head(5)

[Row(_c0='5.1', _c1='3.5', _c2='1.4', _c3='0.2', _c4='1.0'),
 Row(_c0='4.9', _c1='3.0', _c2='1.4', _c3='0.2', _c4='1.0'),
 Row(_c0='4.7', _c1='3.2', _c2='1.3', _c3='0.2', _c4='1.0'),
 Row(_c0='4.6', _c1='3.1', _c2='1.5', _c3='0.2', _c4='1.0'),
 Row(_c0='5.0', _c1='3.6', _c2='1.4', _c3='0.2', _c4='1.0')]

In [8]:
train_data.show(5)


+---+---+---+---+---+
|_c0|_c1|_c2|_c3|_c4|
+---+---+---+---+---+
|5.1|3.5|1.4|0.2|1.0|
|4.9|3.0|1.4|0.2|1.0|
|4.7|3.2|1.3|0.2|1.0|
|4.6|3.1|1.5|0.2|1.0|
|5.0|3.6|1.4|0.2|1.0|
+---+---+---+---+---+
only showing top 5 rows



In [9]:
new_col = train_data['_c0'] * train_data['_c1']

In [10]:
new_train_data =train_data.withColumn('new_col', new_col)

In [11]:
new_train_data

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, new_col: double]

In [12]:
new_train_data.show()

+---+---+---+---+---+------------------+
|_c0|_c1|_c2|_c3|_c4|           new_col|
+---+---+---+---+---+------------------+
|5.1|3.5|1.4|0.2|1.0|17.849999999999998|
|4.9|3.0|1.4|0.2|1.0|14.700000000000001|
|4.7|3.2|1.3|0.2|1.0|15.040000000000001|
|4.6|3.1|1.5|0.2|1.0|             14.26|
|5.0|3.6|1.4|0.2|1.0|              18.0|
|5.4|3.9|1.7|0.4|1.0|21.060000000000002|
|4.6|3.4|1.4|0.3|1.0|15.639999999999999|
|5.0|3.4|1.5|0.2|1.0|              17.0|
|4.4|2.9|1.4|0.2|1.0|             12.76|
|4.9|3.1|1.5|0.1|1.0|15.190000000000001|
|5.4|3.7|1.5|0.2|1.0|19.980000000000004|
|4.8|3.4|1.6|0.2|1.0|             16.32|
|4.8|3.0|1.4|0.1|1.0|14.399999999999999|
|4.3|3.0|1.1|0.1|1.0|12.899999999999999|
|5.8|4.0|1.2|0.2|1.0|              23.2|
|5.7|4.4|1.5|0.4|1.0|25.080000000000002|
|5.4|3.9|1.3|0.4|1.0|21.060000000000002|
|5.1|3.5|1.4|0.3|1.0|17.849999999999998|
|5.7|3.8|1.7|0.3|1.0|             21.66|
|5.1|3.8|1.5|0.3|1.0|             19.38|
+---+---+---+---+---+------------------+
only showing top

In [13]:
show_result = new_train_data.show(5)

+---+---+---+---+---+------------------+
|_c0|_c1|_c2|_c3|_c4|           new_col|
+---+---+---+---+---+------------------+
|5.1|3.5|1.4|0.2|1.0|17.849999999999998|
|4.9|3.0|1.4|0.2|1.0|14.700000000000001|
|4.7|3.2|1.3|0.2|1.0|15.040000000000001|
|4.6|3.1|1.5|0.2|1.0|             14.26|
|5.0|3.6|1.4|0.2|1.0|              18.0|
+---+---+---+---+---+------------------+
only showing top 5 rows



In [14]:
type(show_result)

NoneType

In [15]:
train_data.columns

['_c0', '_c1', '_c2', '_c3', '_c4']

In [16]:
train_data.select('_c0').show(10)

+---+
|_c0|
+---+
|5.1|
|4.9|
|4.7|
|4.6|
|5.0|
|5.4|
|4.6|
|5.0|
|4.4|
|4.9|
+---+
only showing top 10 rows



In [17]:

train_data = train_data.withColumn('_c0', train_data['_c0'].cast('double'))\
       .withColumn('_c1', train_data['_c1'].cast('double'))\
       .withColumn('_c2', train_data['_c2'].cast('double'))\
       .withColumn('_c3', train_data['_c3'].cast('double'))
input_col = ['_c0', '_c1', '_c2', '_c3']
vecAssembler = VectorAssembler(inputCols=input_col, outputCol="features")
new_train_data = vecAssembler.transform(train_data)
# stringIndexer = StringIndexer(inputCol="_c4", outputCol="label")
# pipeline = Pipeline(stages=[vecAssembler, stringIndexer])
# pipelineFit = pipeline.fit(train_data)
# dataset = pipelineFit.transform(train_data)

In [18]:
new_train_data

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector]

In [63]:
new_train_data.head(2)

[Row(_c0=5.1, _c1=3.5, _c2=1.4, _c3=0.2, _c4='1.0', features=DenseVector([5.1, 3.5, 1.4, 0.2])),
 Row(_c0=4.9, _c1=3.0, _c2=1.4, _c3=0.2, _c4='1.0', features=DenseVector([4.9, 3.0, 1.4, 0.2]))]

In [20]:
new_new_train_data = new_train_data.withColumn('label', new_train_data['_c4'].cast('double'))

In [21]:
new_new_train_data

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector, label: double]

In [22]:
new_new_train_data.show(5)

+---+---+---+---+---+-----------------+-----+
|_c0|_c1|_c2|_c3|_c4|         features|label|
+---+---+---+---+---+-----------------+-----+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|  1.0|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|  1.0|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|  1.0|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  1.0|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|  1.0|
+---+---+---+---+---+-----------------+-----+
only showing top 5 rows



In [23]:
train_data.show(5)

+---+---+---+---+---+
|_c0|_c1|_c2|_c3|_c4|
+---+---+---+---+---+
|5.1|3.5|1.4|0.2|1.0|
|4.9|3.0|1.4|0.2|1.0|
|4.7|3.2|1.3|0.2|1.0|
|4.6|3.1|1.5|0.2|1.0|
|5.0|3.6|1.4|0.2|1.0|
+---+---+---+---+---+
only showing top 5 rows



In [24]:
# 新_模型训练
lr_3 = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)
lrModel_3 = lr_3.fit(new_new_train_data)

In [25]:
pre_3 = lrModel_3.transform(new_new_train_data)

In [26]:
pre_3.show(500)

+---+---+---+---+---+-----------------+-----+--------------------+--------------------+----------+
|_c0|_c1|_c2|_c3|_c4|         features|label|       rawPrediction|         probability|prediction|
+---+---+---+---+---+-----------------+-----+--------------------+--------------------+----------+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|  1.0|[-3.5921473185105...|[0.00157155276833...|       1.0|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|  1.0|[-3.5335755527481...|[0.00215601550463...|       1.0|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|  1.0|[-3.5417261943304...|[0.00184854030564...|       1.0|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  1.0|[-3.5284115349670...|[0.00201701076712...|       1.0|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|  1.0|[-3.5966005410309...|[0.00144326216378...|       1.0|
|5.4|3.9|1.7|0.4|1.0|[5.4,3.9,1.7,0.4]|  1.0|[-3.6511035819565...|[0.00135870845931...|       1.0|
|4.6|3.4|1.4|0.3|1.0|[4.6,3.4,1.4,0.3]|  1.0|[-3.5578601196240...|[0.00167224197126...|       1.0|
|5.0|3.4|1

In [31]:
pre_3_test = pre_3.select(['_c0','probability'])

In [49]:
pre_3_test.show()

+---+--------------------+
|_c0|         probability|
+---+--------------------+
|5.1|[0.00157155276833...|
|4.9|[0.00215601550463...|
|4.7|[0.00184854030564...|
|4.6|[0.00201701076712...|
|5.0|[0.00144326216378...|
|5.4|[0.00135870845931...|
|4.6|[0.00167224197126...|
|5.0|[0.00169904494297...|
|4.4|[0.00222866869646...|
|4.9|[0.00199136561308...|
|5.4|[0.00141772976228...|
|4.8|[0.00169445886230...|
|4.8|[0.00207929376476...|
|4.3|[0.00192532173913...|
|5.8|[0.00111332240277...|
|5.7|[9.03078681020208...|
|5.4|[0.00127036007190...|
|5.1|[0.00163043654187...|
|5.7|[0.00145343844284...|
|5.1|[0.00132075328240...|
+---+--------------------+
only showing top 20 rows



In [46]:
pre_3_test.select('probability').collect()[0][0][0]

0.0015715527683337984

In [44]:
def f(df):
    df.withColumn('probb', df.select('probability').collect()[0][0][0])

In [ ]:
def f1(df):
    

In [45]:
pre_3_test.foreach(f)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 32.0 failed 1 times, most recent failure: Lost task 0.0 in stage 32.0 (TID 32, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\sql\types.py", line 1556, in __getattr__
    idx = self.__fields__.index(item)
ValueError: 'withColumn' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 230, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 225, in process
  File "C:\spark\python\pyspark\rdd.py", line 2457, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "C:\spark\python\pyspark\rdd.py", line 2457, in pipeline_func
   

In [27]:
pre_3_new = pre_3.map(lambda x:x.select('probability').collect().toArray())

AttributeError: 'DataFrame' object has no attribute 'map'

In [41]:
pre_3['probability']

Column<b'probability'>

In [42]:
pre_3.select('probability')

DataFrame[probability: vector]

In [55]:
vec = pre_3.select('probability')
vec

DataFrame[probability: vector]

In [56]:
vec = vec.collect()
vec

[Row(probability=DenseVector([0.0016, 0.7914, 0.1504, 0.0567])),
 Row(probability=DenseVector([0.0022, 0.6804, 0.2289, 0.0885])),
 Row(probability=DenseVector([0.0018, 0.7571, 0.1722, 0.0689])),
 Row(probability=DenseVector([0.002, 0.7275, 0.1909, 0.0796])),
 Row(probability=DenseVector([0.0014, 0.8187, 0.1297, 0.0501])),
 Row(probability=DenseVector([0.0014, 0.8193, 0.1258, 0.0536])),
 Row(probability=DenseVector([0.0017, 0.7935, 0.1418, 0.063])),
 Row(probability=DenseVector([0.0017, 0.7704, 0.1641, 0.0638])),
 Row(probability=DenseVector([0.0022, 0.6971, 0.2109, 0.0899])),
 Row(probability=DenseVector([0.002, 0.7149, 0.207, 0.0761])),
 Row(probability=DenseVector([0.0014, 0.8083, 0.1398, 0.0506])),
 Row(probability=DenseVector([0.0017, 0.7795, 0.1552, 0.0636])),
 Row(probability=DenseVector([0.0021, 0.7029, 0.2154, 0.0796])),
 Row(probability=DenseVector([0.0019, 0.763, 0.1688, 0.0664])),
 Row(probability=DenseVector([0.0011, 0.8533, 0.1099, 0.0357])),
 Row(probability=DenseVector([

In [58]:
vec_row = vec[0]

In [60]:
vec_row[0]

DenseVector([0.0016, 0.7914, 0.1504, 0.0567])

In [63]:
vec_row[0][0]

0.0015715527683337984

In [61]:
arr = vec_row[0].toArray()

In [62]:
arr

array([0.00157155, 0.79135577, 0.15035964, 0.05671303])

In [64]:
arr[0]

0.0015715527683337984

In [65]:
pre_3_vec = pre_3.select('probability')
pre_3_vec

DataFrame[probability: vector]

In [73]:
pre_list = pre_3_vec[0].toArray()

TypeError: 'Column' object is not callable

In [66]:
pre_3_vec[0]

Column<b'probability'>

In [67]:
pre_3_vec[0][0]

Column<b'probability[0]'>

In [69]:
pre_3_vec.head()

Row(probability=DenseVector([0.0016, 0.7914, 0.1504, 0.0567]))

In [18]:
new_prob_3 = pre_3.select(['_c0', '_c1', 'probability'])

In [19]:
new_prob_3.show(5)

+---+---+--------------------+
|_c0|_c1|         probability|
+---+---+--------------------+
|5.1|3.5|[0.00157155276833...|
|4.9|3.0|[0.00215601550463...|
|4.7|3.2|[0.00184854030564...|
|4.6|3.1|[0.00201701076712...|
|5.0|3.6|[0.00144326216378...|
+---+---+--------------------+
only showing top 5 rows



In [20]:
new_prob_3

DataFrame[_c0: double, _c1: double, probability: vector]

In [21]:
new_prob_3.probability

Column<b'probability'>

In [22]:
vs = VectorSlicer(inputCol="probability", outputCol="prob_1", indices=[1])
prob_1 = vs.transform(new_prob_3)
prob_1.show(5)

+---+---+--------------------+--------------------+
|_c0|_c1|         probability|              prob_1|
+---+---+--------------------+--------------------+
|5.1|3.5|[0.00157155276833...|[0.7913557718676799]|
|4.9|3.0|[0.00215601550463...|[0.6804221441922835]|
|4.7|3.2|[0.00184854030564...|[0.7571145009018414]|
|4.6|3.1|[0.00201701076712...|[0.7274824434397575]|
|5.0|3.6|[0.00144326216378...|[0.8187212689762617]|
+---+---+--------------------+--------------------+
only showing top 5 rows



In [23]:
prob_1

DataFrame[_c0: double, _c1: double, probability: vector, prob_1: vector]

In [24]:
new_prob_1 = prob_1.select(['_c0', '_c1', 'prob_1'])

In [25]:
new_prob_1.show(5)

+---+---+--------------------+
|_c0|_c1|              prob_1|
+---+---+--------------------+
|5.1|3.5|[0.7913557718676799]|
|4.9|3.0|[0.6804221441922835]|
|4.7|3.2|[0.7571145009018414]|
|4.6|3.1|[0.7274824434397575]|
|5.0|3.6|[0.8187212689762617]|
+---+---+--------------------+
only showing top 5 rows



In [37]:
new_prob_1.select('_c0').collect()

[Row(_c0=5.1),
 Row(_c0=4.9),
 Row(_c0=4.7),
 Row(_c0=4.6),
 Row(_c0=5.0),
 Row(_c0=5.4),
 Row(_c0=4.6),
 Row(_c0=5.0),
 Row(_c0=4.4),
 Row(_c0=4.9),
 Row(_c0=5.4),
 Row(_c0=4.8),
 Row(_c0=4.8),
 Row(_c0=4.3),
 Row(_c0=5.8),
 Row(_c0=5.7),
 Row(_c0=5.4),
 Row(_c0=5.1),
 Row(_c0=5.7),
 Row(_c0=5.1),
 Row(_c0=5.4),
 Row(_c0=5.1),
 Row(_c0=4.6),
 Row(_c0=5.1),
 Row(_c0=4.8),
 Row(_c0=5.0),
 Row(_c0=5.0),
 Row(_c0=5.2),
 Row(_c0=5.2),
 Row(_c0=4.7),
 Row(_c0=4.8),
 Row(_c0=5.4),
 Row(_c0=5.2),
 Row(_c0=5.5),
 Row(_c0=4.9),
 Row(_c0=5.0),
 Row(_c0=5.5),
 Row(_c0=4.9),
 Row(_c0=4.4),
 Row(_c0=5.1),
 Row(_c0=5.0),
 Row(_c0=4.5),
 Row(_c0=4.4),
 Row(_c0=5.0),
 Row(_c0=5.1),
 Row(_c0=4.8),
 Row(_c0=5.1),
 Row(_c0=4.6),
 Row(_c0=5.3),
 Row(_c0=5.0),
 Row(_c0=7.0),
 Row(_c0=6.4),
 Row(_c0=6.9),
 Row(_c0=5.5),
 Row(_c0=6.5),
 Row(_c0=5.7),
 Row(_c0=6.3),
 Row(_c0=4.9),
 Row(_c0=6.6),
 Row(_c0=5.2),
 Row(_c0=5.0),
 Row(_c0=5.9),
 Row(_c0=6.0),
 Row(_c0=6.1),
 Row(_c0=5.6),
 Row(_c0=6.7),
 Row(_c0=5

In [38]:
new_prob_1.select('_c0').collect()[0]

Row(_c0=5.1)

In [39]:
new_prob_1.select('_c0').collect()[0][0]

5.1

In [26]:
type(new_prob_1['prob_1'])

pyspark.sql.column.Column

In [27]:
new_prob_1['prob_1'].cast('string')

Column<b'CAST(prob_1 AS STRING)'>

In [28]:
type(new_prob_1['prob_1'])

pyspark.sql.column.Column

In [29]:
new2 = new_prob_1.select(['_c0', '_c1', new_prob_1['prob_1'].cast('string').alias('prob_1_str')])

In [30]:
new2

DataFrame[_c0: double, _c1: double, prob_1_str: string]

In [31]:
type(new2)

pyspark.sql.dataframe.DataFrame

In [32]:
new2.toPandas().to_csv('C:\Wang Hanmo\scripts\spark\prob_4.csv', index=False)

In [33]:
new_prob_1.select(new_prob_1['prob_1'].cast('string').alias('prob_1_str'))

DataFrame[prob_1_str: string]

In [34]:
new_prob_1

DataFrame[_c0: double, _c1: double, prob_1: vector]

In [35]:
new2_prob_1 = new_prob_1.withColumn('prob_1', new_prob_1['prob_1'].collect()

SyntaxError: unexpected EOF while parsing (<ipython-input-35-666840b5f7c2>, line 1)

In [ ]:
new_prob_1.select('prob_1')[0]

In [ ]:
new_prob_1['prob_1']

In [ ]:
new_prob_1.write.csv('C:\Wang Hanmo\scripts\spark\prob_4.csv')

In [ ]:
prob_pd = new_prob_1.toPandas()

In [ ]:
prob_pd

In [ ]:
prob_pd.to_csv('C:\Wang Hanmo\scripts\spark\prob_3.csv', index=False)

In [ ]:
type(prob_pd['probability'])

In [ ]:
prob_pd['probability'][0]

In [ ]:
type(prob_pd['probability'][0][1])

In [ ]:
def getVal(df):
    return df[0][1]

In [ ]:
new_prob_pd = prob_pd['probability'].apply(getVal)

In [ ]:
prob_pd['prob_1'][0][0]

In [ ]:
type(prob_pd['probability'])

In [ ]:
prob_1.select('prob_1')

In [ ]:
prob_1.prob_1

In [ ]:
prob_1.select('prob_1').collect()[0]

In [ ]:
prob_1.select('prob_1').collect()[0][0]

In [ ]:
prob_1.select('prob_1').collect()[0][0][0]

In [ ]:
type(prob_1.select('prob_1').collect()[0][0][0])

In [ ]:
prob_1_csv = prob_1.select(['_c0', 'prob_1'])

In [ ]:
prob_1_csv

In [ ]:
prob_1_df = prob_1_csv.toPandas()

In [ ]:
prob_1_df['prob_1'][0]

In [ ]:
prob_1_df['prob_1']

In [ ]:
type(prob_1_df['prob_1'])

In [ ]:
prob_1_csv

In [ ]:
prob_1_csv.toPandas().to_csv('C:\Wang Hanmo\scripts\spark\prob_2.csv', index=False)

In [ ]:
prob_1_csv.write.csv('C:\Wang Hanmo\scripts\spark\prob_1.csv')

In [ ]:
new_prob_3 = new_prob_3.withColumn('prob_1', prob_1)

In [ ]:
prob_3

In [ ]:
new_prob_3 = new_prob_3.withColumn('probability', new_prob_3.probability.getItem(0))

In [ ]:
prob_df = new_prob_3.select('probability')

In [ ]:
prob_c2 = pre_3.select('_c2')
prob_c2.show(5)

In [ ]:
new_prob_3 = new_prob_3.withColumn('new_c2', prob_c2.getItem(0))

In [ ]:
prob_df.show(5)

In [ ]:
prob_df[0][0]

In [ ]:
new_prob_3.withColumn('probability', prob_df.getItem(0))

In [ ]:
tra_df = train_data.select(['_c0', '_c1'])
tra_df

In [ ]:
dataset.show(10)

In [ ]:
dataset.select('_c0')

In [ ]:
dataset

In [ ]:
dataset.select('label')

In [ ]:
c_1 = dataset['_c0']
c_1

In [ ]:
col_v1 = dataset._c0
col_v1

In [ ]:
col_v2 = dataset._c1
col_v2

In [ ]:
col_v12 = col_v1 * col_v2
col_v12

In [ ]:
new_dataset = dataset.withColumn('col_v12', col_v12)

In [ ]:
new_dataset.show(10)

In [ ]:
col1 = dataset.select('_c0')
col1_pd = col1.toPandas()
col1_ar = col1_pd.values
col1_ar

In [ ]:
col2 = dataset.select('_c1')
col2_pd = col2.toPandas()
col2_ar = col2_pd.values
col2_ar

In [ ]:
coll = col1_ar * col2_ar
coll

In [ ]:
coll_df = pd.DataFrame(coll)

In [ ]:
type(coll_df)

In [ ]:
coll_spark_df = spark.createDataFrame(coll_df)

In [ ]:
coll_spark_df

In [ ]:
type(coll_spark_df)

In [ ]:
coll_cl = coll_spark_df.collect()

In [ ]:
coll_cl

In [ ]:
coll_list = [i[0] for i in coll_cl]

In [ ]:
coll_list

In [ ]:
lit(0)

In [ ]:
dataset.columns

In [ ]:
dataset.select(['_c0', '_c1']).show(5)

In [ ]:
dataset_delete = dataset.select([i for i in dataset.columns if i != 'label'])

In [ ]:
dataset_delete.show(5)

In [ ]:
dataset._c0

In [ ]:
new_dataset = dataset.withColumn('_c00', dataset['_c0'] + 2)

In [ ]:
new_dataset

In [ ]:
new_dataset.show()

In [ ]:
type(dataset.select('_c0'))

In [ ]:
dataset_cl= dataset.select('_c0').collect()

In [ ]:
dataset_cl

In [ ]:
# dataset_list = [i[0] for i in dataset_cl]  # OK
dataset_list = [i['_c0'] for i in dataset_cl]  # OK

In [ ]:
dataset_list

In [ ]:
type(dataset_list[0])

In [ ]:
# 删除某些无用列
new_dataset = dataset.drop('_c0', '_c1', '_c2', '_c3', '_c4')

In [ ]:
new_dataset.show(5)

In [ ]:
# 新_模型训练
lr_2 = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel_2 = lr_2.fit(new_dataset)

In [ ]:
lrModel_2.save('C:\Wang Hanmo\scripts\spark\lr')

In [ ]:
lr_model = LogisticRegression.load('C:\Wang Hanmo\scripts\spark\lr')

In [ ]:
lr_model = LogisticRegression.load('C:\Wang Hanmo\scripts\spark\lr')

In [ ]:
lr_model = LogisticRegressionModel.load('C:\Wang Hanmo\scripts\spark\lr')

In [ ]:
predictions4 = lr_model.transform(new_dataset, thresholds=0.6)

In [ ]:
predictions4.show(5)

In [ ]:
# 修改阈值
lr_model.params

In [ ]:
lr_model.getParam('threshold')

In [ ]:
lr_model.getParam('thresholds')

In [ ]:
lr_model.isDefined('thresholds')

In [ ]:
lr_model.isSet('thresholds')

In [ ]:
lr_model_read = lr_model.read()

In [ ]:
lr_model_read

In [ ]:
lr_model_read.set('threshold', 0.6)

In [ ]:
lr_model.set(threshold, 0.6)

In [ ]:
lr_model

In [ ]:
prediction_3 = lr_model.transform(new_dataset)

In [ ]:
prediction_3.show()

In [ ]:
prediction_2 = lrModel_2.transform(new_dataset)

In [ ]:
prediction_2.show(150)

In [ ]:
label_list = prediction_2.select('label').collect()

In [ ]:
prob_list = prediction_2.select('probability').collect()

In [ ]:
type(label_list)

In [ ]:
type(label_list[0][0])

In [ ]:
label_list[0][0]

In [ ]:
label_list = [label_list[i][0] for i in range(0, len(label_list))]

In [ ]:
prob_list = prediction_2.select('probability').collect()

In [ ]:
prob_list[0][0]

In [ ]:
prob_list = [prob_list[i][0] for i in range(0, len(prob_list))]

In [ ]:
prob_list


In [ ]:
label_list[0] >= 0.0

In [ ]:
for i in label_list:
    print(type(i))

In [ ]:
trainingSummary = lrModel_2.summary
trainingSummary

In [ ]:
trainingSummary.falsePositiveRateByLabel

In [ ]:
trainingSummary.accuracy

In [ ]:
trainingSummary.precisionByLabel    # 按照0,1,2

In [ ]:
trainingSummary.labels

In [ ]:
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

In [ ]:
float(1)

In [ ]:
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))


In [ ]:
(trainingData, testData) = dataset.randomSplit([0.8, 0.2], 123)
# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))
# trainingData.show(10)
testData.show(5)

In [ ]:
trainingData.show(5)

In [ ]:
# 模型训练
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)

In [ ]:
# 模型预测
predictions = lrModel.transform(testData)
# print(prediction)
predictions.show(10)

# ROC score
# evaluator = BinaryClassificationEvaluator()
# evaluator.evaluate(prediction)

In [ ]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

In [ ]:
accuracy

In [ ]:
preSummary = prediction.summary
preSummary

In [ ]:
preSummary.accuracy

In [ ]:
preSummary.weightedRecall

In [109]:
sentenceDataFrame = spark.createDataFrame((
      (1, "asf"),
      (5, "2143"),
      (3, "rfds")
    )).toDF("label", "sentence")
sentenceDataFrame.show()

sentenceDataFrame1 = spark.createDataFrame((
      (1, "asf"),
      (2, "2143"),
      (4, "rfds")
    )).toDF("label", "sentence")

+-----+--------+
|label|sentence|
+-----+--------+
|    1|     asf|
|    5|    2143|
|    3|    rfds|
+-----+--------+



In [110]:
condition = [sentenceDataFrame.sentence==sentenceDataFrame1.sentence, sentenceDataFrame.label==sentenceDataFrame1.label]
result = sentenceDataFrame.join(sentenceDataFrame1, condition, 'inner')

In [111]:
result.show()


+-----+--------+-----+--------+
|label|sentence|label|sentence|
+-----+--------+-----+--------+
|    1|     asf|    1|     asf|
+-----+--------+-----+--------+



In [60]:
result.count()

2

In [72]:
df = spark.createDataFrame([('100-200',)], ['str'])

In [73]:
df

DataFrame[str: string]

In [74]:
df.show()

+-------+
|    str|
+-------+
|100-200|
+-------+



In [112]:
df_dd = df.select(df.str.cast("float").alias('f'))

In [114]:
df_dd

DataFrame[f: float]

In [115]:
df_dd.show()

+----+
|   f|
+----+
|null|
+----+



In [100]:
df_new = df.select(regexp_replace('str', '-', '').alias('str'))

In [101]:
df_new


DataFrame[str: string]

In [102]:
df_new.show()

+------+
|   str|
+------+
|100200|
+------+



In [91]:
df_df = df_new.select(df_new.d.cast("float"))

In [92]:
df_df

DataFrame[d: float]

In [93]:
df_df.show()

+--------+
|       d|
+--------+
|100200.0|
+--------+

